# 🚀 Pinokio Cloud GPU - Nuclear D-Bus Fix

**AGGRESSIVE FIX for persistent D-Bus errors**

This version uses environment isolation and alternative approaches.

---

In [ ]:
#@title 🔥 Nuclear D-Bus Fix Setup
#@markdown This version uses environment isolation to completely avoid D-Bus

import os
import subprocess
import requests
import time
import json
from pathlib import Path

class PinokioCloudNuclear:
    def __init__(self):
        self.pinokio_path = "/content/pinokio"
        self.binary_path = None
        self.server_process = None
        self.tunnel_process = None
        self.tunnel_url = None
        
    def setup_directory(self):
        print("📁 Setting up directory...")
        Path(self.pinokio_path).mkdir(parents=True, exist_ok=True)
        os.chdir(self.pinokio_path)
        print(f"✅ Created directory: {self.pinokio_path}")
        
    def download_pinokio(self):
        print("📥 Downloading Pinokio...")
        self.binary_path = os.path.join(self.pinokio_path, "Pinokio-linux.AppImage")
        
        if os.path.exists(self.binary_path):
            print("✅ Pinokio already downloaded")
            return True
            
        try:
            download_url = "https://github.com/pinokiocomputer/pinokio/releases/download/3.9.0/Pinokio-3.9.0.AppImage"
            result = subprocess.run(['wget', '-q', '--show-progress', '-O', self.binary_path, download_url], capture_output=True, text=True)
            
            if result.returncode != 0:
                print(f"❌ Download failed: {result.stderr}")
                return False
                
            os.chmod(self.binary_path, 0o755)
            
            if os.path.exists(self.binary_path) and os.access(self.binary_path, os.X_OK):
                print("✅ Pinokio downloaded and configured")
                return True
            else:
                print("❌ Binary not properly configured")
                return False
                
        except Exception as e:
            print(f"❌ Download error: {e}")
            return False
    
    def start_pinokio_server(self):
        print("🔥 Starting Pinokio with NUCLEAR D-Bus isolation...")
        
        if not self.binary_path or not os.path.exists(self.binary_path):
            print("❌ Pinokio binary not found")
            return False
            
        try:
            # NUCLEAR ENVIRONMENT ISOLATION
            env = os.environ.copy()
            env['PINOKIO_APP_PORT'] = '42000'
            
            # COMPLETELY DISABLE D-BUS
            env['DBUS_SESSION_BUS_ADDRESS'] = ''
            env['DBUS_SYSTEM_BUS_ADDRESS'] = ''
            env['NO_DBUS'] = '1'
            env['DISABLE_DBUS'] = '1'
            
            # FORCE XVFB MODE (fake display)
            env['DISPLAY'] = ':99'
            env['XVFB_RUN'] = '1'
            
            # ELECTRON ISOLATION
            env['ELECTRON_DISABLE_SECURITY_WARNINGS'] = 'true'
            env['ELECTRON_NO_ATTACH_CONSOLE'] = '1'
            
            # Start fake X server first
            print("🖥️ Starting virtual display...")
            subprocess.run(['Xvfb', ':99', '-screen', '0', '1024x768x24'], stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL, timeout=3)
            
            print("🚀 Starting isolated server...")
            self.server_process = subprocess.Popen([
                self.binary_path,
                '--no-sandbox',
                '--headless',
                '--disable-dev-shm-usage',
                '--disable-gpu',
                '--disable-gpu-sandbox', 
                '--disable-software-rasterizer',
                '--no-first-run',
                '--disable-default-apps',
                '--disable-extensions',
                '--disable-plugins',
                '--disable-sync',
                '--disable-translate',
                '--disable-background-timer-throttling',
                '--disable-renderer-backgrounding',
                '--disable-backgrounding-occluded-windows',
                '--disable-ipc-flooding-protection',
                '--disable-features=VizDisplayCompositor,TranslateUI,BlinkGenPropertyTrees',
                '--force-color-profile=srgb',
                '--disable-accelerated-2d-canvas',
                '--disable-accelerated-video-decode',
                '--disable-logging',
                '--silent-debugger-extension-api',
                '--disable-component-update',
                '--no-service-autorun',
                '--disable-background-mode'
            ], env=env, stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)
            
            # Wait for server
            print("⏳ Waiting for nuclear startup...")
            for i in range(60):  # Extended timeout
                try:
                    response = requests.get('http://localhost:42000', timeout=3)
                    if response.status_code == 200:
                        print("✅ 🔥 NUCLEAR FIX WORKED! Server running on port 42000")
                        return True
                except requests.exceptions.RequestException:
                    time.sleep(1)
                    if i % 10 == 0 and i > 0:
                        print(f"⏳ Nuclear startup... ({i+1}/60)")
                        
            print("❌ Nuclear startup timeout - checking process...")
            if self.server_process.poll() is not None:
                stdout, stderr = self.server_process.communicate()
                print(f"❌ Process exited with code {self.server_process.returncode}")
                print(f"Error: {stderr[:500]}")
            return False
            
        except Exception as e:
            print(f"❌ Nuclear startup error: {e}")
            return False
    
    def setup_tunnel(self):
        print("🌐 Setting up tunnel...")
        
        try:
            # Try Cloudflare tunnel
            cf_path = '/tmp/cloudflared'
            if not os.path.exists(cf_path):
                print("📥 Downloading Cloudflare tunnel...")
                subprocess.run(['wget', '-q', '-O', cf_path, 'https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64'])
                os.chmod(cf_path, 0o755)
            
            self.tunnel_process = subprocess.Popen([cf_path, 'tunnel', '--url', 'http://localhost:42000'], stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)
            
            print("⏳ Starting tunnel...")
            for _ in range(30):
                line = self.tunnel_process.stdout.readline()
                if 'trycloudflare.com' in line:
                    for part in line.split():
                        if 'trycloudflare.com' in part:
                            self.tunnel_url = part.strip()
                            print(f"✅ Tunnel created: {self.tunnel_url}")
                            return self.tunnel_url
                time.sleep(1)
            
            print("❌ Tunnel setup failed")
            return None
            
        except Exception as e:
            print(f"❌ Tunnel error: {e}")
            return None
    
    def cleanup(self):
        if self.server_process:
            self.server_process.terminate()
        if self.tunnel_process:
            self.tunnel_process.terminate()

# NUCLEAR EXECUTION
try:
    print("🔥 STARTING NUCLEAR D-BUS ISOLATION")
    print("=" * 50)
    
    pinokio = PinokioCloudNuclear()
    
    pinokio.setup_directory()
    
    if not pinokio.download_pinokio():
        raise Exception("Failed to download Pinokio")
    
    if not pinokio.start_pinokio_server():
        raise Exception("Nuclear fix failed - D-Bus still interfering")
    
    tunnel_url = pinokio.setup_tunnel()
    
    print("\n🔥 NUCLEAR SUCCESS! 🔥")
    print("=" * 50)
    
    if tunnel_url:
        print(f"🌍 Access URL: {tunnel_url}")
    else:
        print("🖥️ Local URL: http://localhost:42000")
    
    _pinokio_instance = pinokio
    _pinokio_url = tunnel_url or "http://localhost:42000"
    
except Exception as e:
    print(f"\n💀 NUCLEAR FIX FAILED: {e}")
    print("\n🚨 This means D-Bus is deeply embedded.")
    print("💡 Try alternative: Use Jupyter/JupyterLab instead of Pinokio")
    print("💡 Or run on a different platform (Paperspace, Kaggle, etc.)")